# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [2]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table("InsertTableName", con=engine)
X = df['message']
y = df[df.columns[4:]].values


In [3]:
#X=X[:10]
#y=y[:10]
print(X.shape)
print(y.shape)

(26216,)
(26216, 36)


### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    # Normalize text
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())    

    # Tokenizing text
    words = word_tokenize(text)

    # Removing stop words
    words = [w for w in words if w not in stopwords.words("english")]
    
    # Lemmatizing
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(w) for w in words]    
    
    return words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),    
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5)
print(X_train.shape)
print(y_train.shape)
pipeline.fit(X_train, y_train)

(13108,)
(13108, 36)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)
for i in range(36):
    print (classification_report(y_test[i], y_pred[i]))
    

             precision    recall  f1-score   support

          0       0.97      1.00      0.99        35
          1       0.00      0.00      0.00         1

avg / total       0.95      0.97      0.96        36

             precision    recall  f1-score   support

          0       0.85      1.00      0.92        29
          1       1.00      0.14      0.25         7
          2       0.00      0.00      0.00         0

avg / total       0.88      0.83      0.79        36

             precision    recall  f1-score   support

          0       0.79      1.00      0.89        27
          1       1.00      0.22      0.36         9

avg / total       0.85      0.81      0.75        36

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        33
          1       1.00      1.00      1.00         3

avg / total       1.00      1.00      1.00        36

             precision    recall  f1-score   support

          0       1.00      1.00 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [8]:
parameters =  {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 100]
        #'clf__estimator__min_samples_split': [2, 4]
    }

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=2)

cv.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[CV] clf__estimator__n_estimators=50, tfidf__use_idf=True ............
[CV]  clf__estimator__n_estimators=50, tfidf__use_idf=True, total= 2.6min
[CV] clf__estimator__n_estimators=50, tfidf__use_idf=True ............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.4min remaining:    0.0s


[CV]  clf__estimator__n_estimators=50, tfidf__use_idf=True, total= 2.6min
[CV] clf__estimator__n_estimators=50, tfidf__use_idf=True ............
[CV]  clf__estimator__n_estimators=50, tfidf__use_idf=True, total= 2.6min
[CV] clf__estimator__n_estimators=50, tfidf__use_idf=False ...........
[CV]  clf__estimator__n_estimators=50, tfidf__use_idf=False, total= 2.6min
[CV] clf__estimator__n_estimators=50, tfidf__use_idf=False ...........
[CV]  clf__estimator__n_estimators=50, tfidf__use_idf=False, total= 2.6min
[CV] clf__estimator__n_estimators=50, tfidf__use_idf=False ...........
[CV]  clf__estimator__n_estimators=50, tfidf__use_idf=False, total= 2.6min
[CV] clf__estimator__n_estimators=100, tfidf__use_idf=True ...........
[CV]  clf__estimator__n_estimators=100, tfidf__use_idf=True, total= 4.2min
[CV] clf__estimator__n_estimators=100, tfidf__use_idf=True ...........
[CV]  clf__estimator__n_estimators=100, tfidf__use_idf=True, total= 4.3min
[CV] clf__estimator__n_estimators=100, tfidf__use_i

[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed: 51.6min finished


GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=2)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:

y_pred = cv.predict(X_test)
for i in range(36):
    print (classification_report(y_test[i], y_pred[i]))
    

             precision    recall  f1-score   support

          0       0.97      1.00      0.99        35
          1       0.00      0.00      0.00         1

avg / total       0.95      0.97      0.96        36

             precision    recall  f1-score   support

          0       0.83      1.00      0.91        29
          1       1.00      0.14      0.25         7

avg / total       0.86      0.83      0.78        36

             precision    recall  f1-score   support

          0       0.79      1.00      0.89        27
          1       1.00      0.22      0.36         9

avg / total       0.85      0.81      0.75        36

             precision    recall  f1-score   support

          0       1.00      1.00      1.00        33
          1       1.00      1.00      1.00         3

avg / total       1.00      1.00      1.00        36

             precision    recall  f1-score   support

          0       1.00      0.97      0.98        30
          1       0.86      1.00 

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [10]:
#-

### 9. Export your model as a pickle file

In [11]:
from sklearn.externals import joblib
joblib.dump(cv.best_estimator_, 'best.pkl', compress = 1)

['best.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.